In [ ]:
import pandas as pd
import numpy as np
from farasa.segmenter import FarasaSegmenter
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import evaluate
from lime.lime_text import LimeTextExplainer
import torch

# Load and prepare dataset
df = pd.read_csv("dataset.csv")
df = df.rename(columns={"Comment": "text", "Toxicity": "label"}) 
df["label"] = df["label"].astype(int)

# Farasa segmentation 
farasa_segmenter = FarasaSegmenter(interactive=True)

def farasa_tokenize(example):
    segmented = farasa_segmenter.segment(example["text"])
    return {"text": segmented}

dataset = Dataset.from_pandas(df)
dataset = dataset.map(farasa_tokenize)

# Train/test split
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']

# Load MARBERTv2
model_name = "UBC-NLP/MARBERTv2"
hf_tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenization 
def tokenize_function(examples):
    return hf_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

# TrainingArguments 
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    #load_best_model_at_end=True,
    logging_dir='./logs',
    metric_for_best_model="accuracy"
)

# Trainer 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=hf_tokenizer,
    compute_metrics=compute_metrics
)

# Train
trainer.train()

# LIME Explanation

# Prediction function for LIME
class_names = ['Non-Toxic', 'Toxic']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict_proba(texts):
    inputs = hf_tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    return probs.cpu().numpy()

# Take a sample sentence
sample_text = df['text'].iloc[0]
print("Sample text:", sample_text)

# LIME explanation
explainer = LimeTextExplainer(class_names=class_names)
explanation = explainer.explain_instance(sample_text, predict_proba, num_features=10)
explanation.show_in_notebook(text=sample_text)  

for feature, weight in explanation.as_list():
    print(f"{feature}: {weight:.4f}")
